In [ ]:
from datetime import timedelta
import matplotlib.pyplot as plt
# import quandl 
import datetime as dt 
import pandas_datareader as pdr 
import pandas as pd 
import numpy as np 
# quandl.ApiConfig.api_key = ''


class portfolio():
    
    def __init__(self, stocks, tickers, start, end ):
        self.num_stock = stocks 
        self.tickers = tickers
        self.start = start 
        self.end = end 
        self.df_close, self.new_tickers = self.get_stock_price()
        self.df_excess, self.df_return = self.excess_return()
        self.covariance_mat = self.variance_covariance_matrix()
        
    def get_stock_price(self):
        df_close = pd.DataFrame([])
        new_tickers = [] 
        for ticker in self.tickers: 
            try:
                df = pdr.DataReader(ticker,'yahoo', start = self.start , end = self.end)
                df_close[ticker] = df['Adj Close']
                new_tickers.append(ticker)  
            except KeyError:
                pass 
        df_close.dropna(inplace = True)
        stock_num = len(df_close.columns)
        return df_close, new_tickers

    def excess_return(self):
        df_excess = pd.DataFrame([])
        df_return = pd.DataFrame([])
        for ticker in self.new_tickers:
            df_return[ticker] = np.log(self.df_close[ticker]).diff()
            df_excess[ticker] = [(i - df_return[ticker].mean(axis = 0)) for i in df_return[ticker].values] #(df.shape[0],ticker)
        df_return.dropna(inplace = True)
        df_excess.dropna(inplace = True)
        return df_excess, df_return  

    def variance_covariance_matrix(self):   
        n = self.df_close.shape[0]
        covariance_matrix = (np.dot((np.array(self.df_excess)/n).T, np.array(self.df_excess))) #(df.shape[0],ticker).T(df.shape[0],ticker) = (ticker,ticker)
        return covariance_matrix #(4,4)

    def std_each_stock(self):
        std = []
        for ticker in self.tickers: 
            a = np.std(np.array(self.df_return[ticker])) #, dtype = np.float64
            std.append(a)
        std_array = np.array(std).reshape(len(self.tickers),1)
        #print("std_array:\n",std_array)
        return std_array 

    def std_matrix(self): 
        std_array = self.std_each_stock() 
        std_mat = np.dot(std_array, std_array.T) #(4,1)(4,1).T = (4,4)
        return std_mat #(stocks, stocks)

    def correlation_matrix(self, weights):
        std_mat = self.std_matrix()
        correlation_mat = self.covariance_mat/std_mat #(4,4)/(4,4)
        
        #print("correlation_mat:\n",correlation_mat)
        return correlation_mat #(4,4)

    def portfolio_risk(self,weights): #Portfolio Variance = Sqrt (Transpose (Wt.SD) * Correlation Matrix * Wt. SD)
        correlation_mat = self.correlation_matrix(weights)
        port_variance = np.dot(np.dot(weights.T, correlation_mat), weights) #{[(4,1).T(4,4) = (1,4)], (4,1)} = (1,1)
        port_std = np.sqrt(port_variance)
        return port_std #(1,1)                   

    def portfolio_return(self,weights):
        average_return_stock ={}
        list_avg_return = []
        avg_return = np.array([])
        for i in self.df_return.columns:
            average_return_stock[i] = self.df_return[i].mean() 
            list_avg_return.append(average_return_stock[i])
        avg_return= np.array(list_avg_return).reshape(1,len(self.df_return.columns))
        expected_return = np.sum(avg_return*weights.T) # (1, ticker)(ticker, 1).T = (df.shape, ticker)
        return expected_return 
    
    def find_current_rf_rate(self):
        end = dt.date.today()
        df = pdr.DataReader('DGS10','fred', start =end, end = end )
        for i in range(30):
            if df.empty:
                end = end + timedelta(-1) 
                df = pdr.DataReader('DGS10','fred', start = end, end = end )
            else:
                print("the US 10 year treasury bond yield is:\n", df.head())
                break 
        return df.values
    
    def Max_sharpe_ratio(self, len_train): # (average return - rf ) /std 
        rf = self.find_current_rf_rate()
        column = ['sharpe', 'return'] + ['weight_{}'.format(i) for i in self.tickers]
        df_risk_return = pd.DataFrame(columns = column, index = [i for i in range(len_train)]) 
        j = 1
        for i in range(0,len_train): 
            k = self.weights_simulation() #(4,1)    
            avg_return = self.portfolio_return(k) #(1,1)
            sharpe = (avg_return - rf)/self.portfolio_risk(k)#[(1,1) - (1,1)]/(1,1)
            df_risk_return.iloc[i,0] = sharpe
            df_risk_return.iloc[i,1] = avg_return
            df_risk_return.iloc[i,2:] = list(k) 
            print("run {}th time:\n sharpe = {} and avg_return = {}\n".format(j,sharpe,avg_return)) 
            j+=1
        print(df_risk_return)
        print("the max sharpe: {}\nand the max return: {}\n".format(df_risk_return['sharpe'].max(),df_risk_return['sharpe'].max()))
        g= np.array(df_risk_return)
        print("len(list(g)):",len(list(g)))
        k = np.argmax(g)
        print("k: ", k)
        print("the corresponding weight of highest sharpe: ", df_risk_return.iloc[k, 2:])
        
        axes = plt.gca()
        axes.set_ylim([-0.004,0.004])
        plt.scatter(list(df_risk_return['sharpe'].values), list(df_risk_return['return'].values))
        return 
    
    def weights_simulation(self):
        b =np.random.randint(10, size = self.num_stock)
        init_weights = np.array([j/np.sum(b) for j in b]).reshape(self.num_stock,1)  #(4,1)  
        return init_weights

    
    
if __name__ == "__main__":
    start = input("please enter start date with format %Y-%m-%d: ")
    end = dt.date.today()
    tickers = []
    stocks = 4 

    for i in range(stocks):  
        num_form = ['st', 'nd','rd','th']
        if i == 0:    
            ticker = input("Enter {}{} ticker for your portfolio:".format(i+1,num_form[i]))
            tickers.append(ticker)
        if i == 1: 
            ticker = input("Enter {}{} ticker for your portfolio:".format(i+1,num_form[i]))
            tickers.append(ticker)
        elif i == 2:
            ticker = input("Enter {}{} ticker for your portfolio:".format(i+1,num_form[i]))
            tickers.append(ticker)
        elif i > 2:
            ticker = input("Enter {}{} ticker for your portfolio:".format(i+1,num_form[i]))
            tickers.append(ticker)

    k = portfolio( stocks = stocks, tickers = tickers, start = start, end = end)
    k.Max_sharpe_ratio(1000)
